### 1. Foremost, import the required libraries.

In [1]:
import pandas as pd

### 2. Check the working directory and it's content.

In [2]:
pwd

'C:\\Users\\Wil\\Desktop\\Guelph\\Thesis\\Data\\Full_Send'

In [3]:
ls

 Volume in drive C has no label.
 Volume Serial Number is DCDB-8A37

 Directory of C:\Users\Wil\Desktop\Guelph\Thesis\Data\Full_Send

2022-03-12  01:55 PM    <DIR>          .
2022-03-12  01:55 PM    <DIR>          ..
2022-03-12  10:22 AM    <DIR>          .ipynb_checkpoints
2022-03-12  01:55 PM            12,763 Full_Send to Individual_Stocks.ipynb
2022-03-11  10:07 PM       366,643,634 Full_Send.csv
2022-03-12  01:54 PM    <DIR>          parsed_data
               2 File(s)    366,656,397 bytes
               4 Dir(s)  380,288,872,448 bytes free


### 3. Pull the data using pandas.

In [4]:
dataset = pd.read_csv("Full_Send.csv")

### 4. Take a quick look at the import.

In [5]:
dataset.head()

,GVKEY,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,curcdq,...,capxy,dvy,prstkcy,costat,cshtrq,prccq,prchq,prclq,gsector,idbflag
0,1000,31/12/1970,1970,4,INDL,C,D,STD,AE.2,USD,...,NaN,NaN,NaN,I,95900.0,10.000,10.875,7.500,NaN,D
1,1000,31/03/1971,1971,1,INDL,C,D,STD,AE.2,USD,...,NaN,NaN,NaN,I,485300.0,9.750,12.875,8.625,NaN,D
2,1000,30/06/1971,1971,2,INDL,C,D,STD,AE.2,USD,...,NaN,NaN,NaN,I,280100.0,8.250,11.375,7.125,NaN,D
3,1000,30/09/1971,1971,3,INDL,C,D,STD,AE.2,USD,...,NaN,NaN,NaN,I,148400.0,4.625,8.125,4.625,NaN,D
4,1000,31/12/1971,1971,4,INDL,C,D,STD,AE.2,USD,...,NaN,NaN,NaN,I,209400.0,5.750,6.375,3.625,NaN,D


### 5.  Count unique companies in dataset

In [6]:
company_count = dataset['tic'].value_counts()
print(company_count)

CRS      242
GIS      242
PG       242
EMR      241
CPB      241
        ... 
LITL       1
VIEW       1
KPLNF      1
FSTC.      1
CIFR       1
Name: tic, Length: 25807, dtype: int64


The length of the series indicates that there are 25,807 unique companies listed in the data. We can also observe that the count ranges from 242 to 1. Given the nature of this data (which is quarterly fundamental and technical data), we can assert that the companies with 242 count correspond to 60 and a half years of data, while the companies with 1 count correspond to a quarter year of data.

### 6. Create sub datasets for each company

We can now create a list object of all of these unique companies, which can be iterated through to create separate time series for each company. This is important for when we train a model. The data needs to be separated as having the time series loop from one company to the next is erronous and will create a poor model.

In [7]:
companies_list = company_count.index.tolist()

The below function allows us to pass a company and extract it out of the full dataset we have above. Following this, we write it to our working directory as a csv file. 

In [10]:
def write_dataset_to_file(company, full_dataset = dataset):
    """
    company (str): The company which is being parsed from the full dataset
    full_dataset (pd.Dataframe): The dataset which is being parsed
    """
    
    sub_dataset = dataset[dataset.tic == company]
    sub_dataset.to_csv('parsed_data/' + company + ".csv")

Now that we have that function defined, we loop through the companies list and separate our data.

In [11]:
for company in companies_list: write_dataset_to_file(company)

### 7. We want to inspect our data to determine what is missing more often, possibly removing features that are too sparse.

In [13]:
pd.set_option('display.max_columns', None)

In [32]:
dataset.describe()

,GVKEY,fyearq,fqtr,actq,apq,atq,ceqq,cheq,cogsq,cstkq,dlcq,dlttq,dpq,epsfiq,epsfxq,ibq,icaptq,invtq,lctq,lltq,niq,nopiq,oibdpq,ppentq,pstknq,pstkrq,rectq,req,revtq,wcapq,xoprq,xrdq,xsgaq,capxy,dvy,prstkcy,cshtrq,prccq,prchq,prclq,gsector
count,1.176169e+06,1.176169e+06,1.176169e+06,810860.000000,9.678790e+05,9.922230e+05,1.014706e+06,9.817040e+05,1.000828e+06,978566.000000,949084.000000,1.008764e+06,867714.000000,1.037232e+06,1.036937e+06,1.074961e+06,1.017479e+06,954710.000000,816975.000000,753441.000000,1.071783e+06,1.007963e+06,878260.000000,964697.000000,1.007494e+06,943585.000000,9.522920e+05,973618.000000,990526.000000,808843.000000,998921.000000,321048.000000,805706.000000,793238.000000,796821.000000,754966.000000,1.136463e+06,1.148331e+06,1.146816e+06,1.146819e+06,1.055050e+06
mean,3.877897e+04,1.997183e+03,2.497485e+00,838.583920,2.169985e+03,6.883034e+03,1.166987e+03,7.847975e+02,3.897584e+02,136.547236,597.907631,1.140106e+03,34.747972,3.024352e-01,2.854263e-01,3.239442e+01,2.322306e+03,404.790831,622.727680,972.636324,3.277785e+01,-3.698535e-01,112.159728,963.968423,2.130905e+01,4.409207,2.133159e+03,632.993852,551.032009,214.550542,470.665325,34.314288,99.588685,104.494917,45.191880,50.114602,2.820360e+07,2.532521e+01,2.855660e+01,2.188232e+01,3.197138e+01
std,5.444968e+04,1.385540e+01,1.116461e+00,4423.625017,3.478871e+04,6.911028e+04,7.043212e+03,1.190084e+04,2.440507e+03,1329.907717,9350.494273,1.145337e+04,232.710427,2.194101e+01,1.725402e+01,3.674216e+02,1.612308e+04,7012.331025,3774.552712,5526.287767,3.877410e+02,2.039157e+02,701.756171,6026.351652,4.854649e+02,70.608973,2.897489e+04,6278.217927,3110.695537,1623.947080,2749.191450,232.222625,542.147545,744.417007,331.922411,572.019456,1.826150e+08,5.784293e+02,6.326206e+02,5.250608e+02,1.263351e+01
min,1.000000e+03,1.961000e+03,1.000000e+00,-0.060000,-4.030000e+00,-6.333300e+01,-1.063620e+05,-3.880000e+01,-1.283350e+04,-118.507000,-1068.000000,-4.501500e+01,-686.987000,-2.973000e+03,-2.973000e+03,-6.165900e+04,-6.167400e+04,-2.700000,-2.870000,-5579.825000,-6.165900e+04,-3.145901e+04,-59740.000000,-0.560000,-2.520000e+02,-372.131000,-4.100000e+01,-143336.328000,-25623.000000,-56429.000000,-12833.503000,-81.517000,-923.000000,-3258.000000,-14.958000,-395.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+01
25%,7.131000e+03,1.987000e+03,2.000000e+00,17.518000,2.317000e+00,4.524300e+01,1.736800e+01,2.894000e+00,4.157000e+00,0.091000,0.102000,5.600000e-01,0.270000,-3.000000e-02,-3.000000e-02,-2.330000e-01,2.797300e+01,0.020000,6.990000,1.727000,-2.660000e-01,0.000000e+00,0.293000,4.575000,0.000000e+00,0.000000,4.761000e+00,-9.661000,7.295000,3.425000,7.122000,0.027000,2.415000,0.320000,0.000000,0.000000,3.784000e+05,5.250000e+00,6.750000e+00,4.250000e+00,2.000000e+01
50%,1.389400e+04,1.998000e+03,2.000000e+00,74.367000,1.305700e+01,2.385990e+02,8.071000e+01,1.821100e+01,2.169800e+01,0.800000,2.819000,2.000000e+01,1.473000,1.600000e-01,1.500000e-01,1.063000e+00,1.320490e+02,6.750000,31.688000,19.895000,1.082000e+00,4.800000e-02,4.708000,28.333000,0.000000e+00,0.000000,2.922150e+01,11.386000,37.102000,25.906000,31.762000,1.491000,8.780000,2.720000,0.000000,0.000000,1.985500e+06,1.331000e+01,1.530000e+01,1.125000e+01,3.500000e+01
75%,3.109900e+04,2.008000e+03,3.000000e+00,328.133750,1.082835e+02,1.380272e+03,3.943700e+02,9.850925e+01,1.208210e+02,11.710000,26.424000,2.258013e+02,9.309000,4.600000e-01,4.500000e-01,8.898000e+00,7.045310e+02,55.984000,170.665000,230.164000,9.058000e+00,6.990000e-01,31.900000,207.915000,0.000000e+00,0.000000,1.949892e+02,121.797000,190.809250,121.920500,159.846000,7.979000,36.294000,20.400750,3.274000,0.561000,1.184705e+07,2.612500e+01,2.950000e+01,2.241000e+01,4.000000e+01
max,3.495300e+05,2.022000e+03,4.000000e+00,207696.000000,2.462303e+06,3.879172e+06,5.061990e+05,1.058786e+06,1.890810e+05,109760.000000,614237.411000,3.048499e+06,28089.000000,1.367900e+04,1.339700e+04,4.784023e+04,3.040128e+06,620828.882000,3297

From here we can determine how much data is missing from the respective features, by dividing the count for each feature by the total number of rows. We'll do this below, and also sort in descending order to be able to view which features are most complete.

In [36]:
(dataset.describe().loc['count', :]/len(dataset)).sort_values(ascending=False)

GVKEY      1.000000
fyearq     1.000000
fqtr       1.000000
prccq      0.976332
prclq      0.975046
prchq      0.975044
cshtrq     0.966241
ibq        0.913951
niq        0.911249
gsector    0.897022
epsfiq     0.881873
epsfxq     0.881622
icaptq     0.865079
ceqq       0.862721
dlttq      0.857669
nopiq      0.856988
pstknq     0.856589
cogsq      0.850922
xoprq      0.849301
atq        0.843606
revtq      0.842163
cheq       0.834662
cstkq      0.831994
req        0.827788
apq        0.822908
ppentq     0.820203
invtq      0.811712
rectq      0.809656
dlcq       0.806928
pstkrq     0.802253
oibdpq     0.746712
dpq        0.737746
lctq       0.694607
actq       0.689408
wcapq      0.687693
xsgaq      0.685026
dvy        0.677472
capxy      0.674425
prstkcy    0.641886
lltq       0.640589
xrdq       0.272961
Name: count, dtype: float64

It's important to note that in particular the xrdq feature is very sparse, with only ~27.3% data present. Evidently, this feature can be eliminated. This raises the question, what is this feature? Actually, it raises the question what is each feature here? This is listed below...

prccq - Quarterly Closing Price  
prclq - Quarterly Low Price  
prchq - Quarterly High Price  
cshtrq - Quarterly Common Shares Traded  
ibq - Quarterly Income Before Extraordinary Items  
niq - Quarterly Net Income (Loss)  
gsector - GIC Sector  
epsfiq - Earnings Per Share (Diluted) - Including Extraordinary Items  
epsfxq - Earnings Per Share (Diluted) - Excluding Extraordinary Items  
icaptq - Quarterly Total Invested Capital  
ceqq - Total Common/Ordinary Equity  
dlttq - Total Long Term Debt  
nopiq - Total Quarterly Non-Operating Income (Expense)  
pstknq - Nonredeemable Preferred/Preference Stock  
cogsq - Quarterly Cost of Goods Sold  
xoprq - Total Quarterly Operating Expense  
atq - Total Assets  
revtq - Total Quarterly Revenue  
cheq - Cash and Short Term Investments  
cstkq - Common/Ordinary Stock (Capital)   
req - Retained Earnings  
apq - Trade Accounts Payable/Creditors  
ppentq - Property, Plant, and Equipment Net Total  
invtq - Total Inventories  
rectq - Total Receivables  
dlcq - Debt in Current Liabilities  
pstkrq - Redeemable Preferred/Preference Stock  
oibdpq - Quarterly Operating Income Before Depreciation  
dpq - Total Depreciation and Amortization  
lctq - Total Current Liabilities  
actq - Total Current Assets  
wcapq - Working Capital (Balance Sheet)  
xsqap - Selling, General and Administrative Expenses  
dvy - Cash Dividends  
capxy - Capital Expenditures  
prstkcy - Purchase of Common and Preferred Stock  
lltq - Total Long Term Liabilities  
xrdq - Research and Development Expense

I am now consulting with my advisors to determine if there are factors I missed or redundant factors.